# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [6]:
#upgrade the sdk version by running below command in jupyter notebook terminal window 
#and restart the kernal before running this command.

#pip install --upgrade azureml-core
#pip install --upgrade azureml-automl-core

SyntaxError: invalid syntax (<ipython-input-6-36cc90423c67>, line 4)

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset



# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')



quick-starts-ws-135700
aml-quickstarts-135700
southcentralus
9a7511b8-150f-4a58-8528-3e7d50216c31


In [3]:
# choose a name for experiment
experiment_name = 'capstone-Automl'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
capstone-Automl,quick-starts-ws-135700,Link to Azure Machine Learning studio,Link to Documentation


# Creat Compute Cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException


# Choose a name for your GPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().
compute_target.get_status()

Creating
Succeeded................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


Data

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory


In [6]:
datastore= ws.get_default_datastore()

In [7]:
datastore.upload(src_dir= './', target_path= './')

Uploading an estimated of 9 files
Target already exists. Skipping upload for automl.ipynb
Target already exists. Skipping upload for data.json
Target already exists. Skipping upload for heart_failure_clinical_records_dataset.csv
Target already exists. Skipping upload for score.py
Target already exists. Skipping upload for .ipynb_checkpoints/automl-checkpoint.ipynb
Uploading ./automl.log
Uploaded ./automl.log, 1 files out of an estimated total of 8
Uploading ./automl_errors.log
Uploaded ./automl_errors.log, 2 files out of an estimated total of 8
Uploading ./azureml_automl.log
Uploaded ./azureml_automl.log, 3 files out of an estimated total of 8
Uploading ./.ipynb_checkpoints/score-checkpoint.py
Uploaded ./.ipynb_checkpoints/score-checkpoint.py, 4 files out of an estimated total of 4
Uploaded 4 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [8]:
tds= Dataset.Tabular.from_delimited_files(datastore.path('./heart_failure_clinical_records_dataset.csv'))

In [9]:
tds.to_pandas_dataframe().head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [10]:
dataset_x = tds.to_pandas_dataframe().drop('DEATH_EVENT', axis= 1)
dataset_x.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [11]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=tds,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
# Retrieve and save your best automl model.
remote_run.wait_for_completion(show_output=True)




Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

*************************************************

{'runId': 'AutoML_7ac4583d-fd6b-4259-9f42-e24ed19f8fd8',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-22T15:51:40.381439Z',
 'endTimeUtc': '2021-01-22T16:17:06.081758Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-Automl","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-135700","workspace_name":"quick-starts-ws-135700","region":"southcentralus","compute_target":"auto-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featuri

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [16]:
#TODO: Save the best model
best_run, fitted_model = remote_run.get_output()


Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


AttributeError: Can't get attribute 'XGBoostLabelEncoder' on <module 'xgboost.compat' from '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost/compat.py'>

In [17]:
print(best_run)
print(fitted_model)

NameError: name 'best_run' is not defined

In [18]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
description = 'HearFailure_AutoML_Model'
model = remote_run.register_model(description = description)


In [20]:
print(model.name, model.id, model.version, model.description, sep ='\t')

AutoML7ac4583df36	AutoML7ac4583df36:2	2	HearFailure_AutoML_Model


In [23]:
import joblib

joblib.dump(model, 'HearFailure_AutoML_Model')

TypeError: can't pickle _thread.RLock objects

In [25]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='HearFailure_AutoML_Model',                # Name of the registered model in your workspace.
                       model_path='./HearFailure_AutoML_Model.pkl',  # Local file to upload and register as a model.
                       description='Geart Failure death prediction',
                       tags={'area': 'heart failure', 'type': 'Classification'})

print('Name:', model.name)
print('Version:', model.version)

ERROR:azureml.core.model:Error, provided model path "./HearFailure_AutoML_Model.pkl" cannot be found



WebserviceException: WebserviceException:
	Message: Error, provided model path "./HearFailure_AutoML_Model.pkl" cannot be found
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error, provided model path \"./HearFailure_AutoML_Model.pkl\" cannot be found"
    }
}

In [22]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


env = Environment.get(ws, "AzureML-AutoML-GPU")
env.inferencing_stack_version='latest'

#for pip_package in ["scikit-learn"]:
#    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

In [34]:
inference_config

InferenceConfig(entry_script=score.py, runtime=None, conda_file=None, extra_docker_file_steps=None, source_directory=None, enable_gpu=None, base_image=None, base_image_registry=<azureml.core.container_registry.ContainerRegistry object at 0x7f77a1dd0c88>)

In [35]:
from azureml.core.webservice import AciWebservice 
from azureml.core import Model

dep_cfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1) 

webservice = Model.deploy(ws, "capstone-automl", [model], inference_config, dep_cfg)

webservice.wait_for_deployment(show_output = True) 
print(webservice.state)  
print(webservice.scoring_uri) 
print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................................................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 903c3a7c-b846-4137-a8c0-a13786476c46
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: capstone-automl. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image viennaglobal.azurecr.io/azureml/azureml_6b386f93e6a650a8cca5338d7fb60982 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for 

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 903c3a7c-b846-4137-a8c0-a13786476c46
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: capstone-automl. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image viennaglobal.azurecr.io/azureml/azureml_6b386f93e6a650a8cca5338d7fb60982 locally. Please refer to https://aka.ms/debugimage#service-launch

In [26]:
webservice.get_logs()

NameError: name 'service' is not defined

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json


input_payload = json.dumps({
    'data': dataset_x[0:2].tolist(),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = webservice.run(input_payload)
print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [28]:
webservice.delete()

NameError: name 'webservice' is not defined